## Ziele:
- Mit Tor Netzwerk verbinden
- fhnw studenthub einloggen
- ects laden
- mail benachrichtigung Änderung
- (modul heruasfinden)

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import subprocess

#import slack_notifications as slack
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import random

import os
import time
import json

In [2]:
# credential file must be present and correct
credentials = json.loads(open("fhnw.json").read())
fhnw_username = credentials['username']
fhnw_password = credentials['password']
email_sender = credentials['email_sender']
email_receiver = credentials['email_receiver']
email_password = credentials['email_password']
tor_path = credentials['tor_path']
repeat = credentials['repeat']
# slack_token = credentials['slack_access']
# slack_channel = credentials['slack_channel']
# slack.ACCESS_TOKEN = slack_token
# slack_username = "studenthub_bot"

try:
    prev_data = json.loads(open("studenthub.json").read())
    prev_credits = prev_data['credits']
except:
    prev_data = {}
    prev_credits = 0
    prev_modules = {}

# dict to save newly loaded studenthub data
data = {}

In [3]:

def send_mail(subject, message):
    server=smtplib.SMTP_SSL("smtp.zoho.com", 465)
    # smtp_server.ehlo()
    #smtp_server.starttls()
    server.login(email_sender, email_password)
    #writing the message in HTML

    msg = MIMEMultipart()
    msg['From'] = email_sender
    msg['To'] = email_receiver
    msg['Subject'] = subject
    body = message
    msg.attach(MIMEText(body, 'plain'))

    email_text = msg.as_string()
    server.sendmail(email_sender, email_receiver, email_text)
    # quit mail server
    server.quit()


In [4]:
buffer_timeout = 10
page_load_timeout = 30
max_retry = 3
retry = 0
last_error = ""

while(True):
    try:
        profile_path = f'{tor_path}\\Browser\\TorBrowser\\Data\\Browser\\profile.default'
        # needed to be done to work
        os.startfile(f'{tor_path}\\Browser\\TorBrowser\\Tor\\tor.exe')
        time.sleep(buffer_timeout)

        options = Options()
        options.set_preference('profile', profile_path)
        options.set_preference('network.proxy.type', 1)
        options.set_preference('network.proxy.socks', '127.0.0.1')
        options.set_preference('network.proxy.socks_port', 9050)
        options.set_preference("network.proxy.socks_remote_dns", False)
        #binary = FirefoxBinary(rf'{tor_path}\Browser\firefox.exe')
        options.binary_location = f'{tor_path}\\Browser\\firefox.exe'

        service = Service(r'./geckodriver.exe')

        driver = webdriver.Firefox(service=service, options=options)

        time.sleep(buffer_timeout)
        driver.get("https://studenthub.technik.fhnw.ch/")

        #wait for browser to start and page to load
        fhnw_login = WebDriverWait(driver, page_load_timeout * 2).until(EC.presence_of_element_located((By.ID, 'fhnwLoginLink')))
        # fix: Message: Element <a id="fhnwLoginLink" class="btn-sts-login" href="{{provider.href}}"> could not be scrolled into view
        time.sleep(buffer_timeout)
        #login
        driver.find_element(By.ID, 'fhnwLoginLink').click()
        username = WebDriverWait(driver, page_load_timeout).until(EC.presence_of_element_located((By.ID, 'username')))
        time.sleep(buffer_timeout)
        driver.find_element(By.ID, 'username').send_keys(fhnw_username)
        driver.find_element(By.ID, 'password').send_keys(fhnw_password)
        driver.find_element(By.NAME, "_eventId_proceed").click()

        # get credits
        credits = WebDriverWait(driver, page_load_timeout).until(EC.presence_of_element_located((By.CLASS_NAME, 'studiengang-credits')))
        time.sleep(buffer_timeout)
        credits_div = driver.find_element(By.CLASS_NAME, 'studiengang-credits')
        credits_text = credits_div.get_attribute("innerText")
        curr_credits = credits_text.split("/")[0]

        data['credits'] = curr_credits

        passed_modules = driver.find_elements(By.CLASS_NAME, 'grade-module')

        delta_text = ""
        modules = {}

        for passed_module in passed_modules:
            grade = passed_module.get_attribute("innerText")
            # returns <Name>\n<Grade>
            name = passed_module.find_element(By.XPATH, "./..").get_attribute("innerText").split("\n")[0]
            modules[name] = grade
            if((not name in prev_data) or (modules[name] != prev_data[name])):
                if(grade != ""):
                    delta_text += f"Modul <{name}> wurde neu mit der Note {grade} bewertet. \r\n"

        # append credits to data dict
        data.update(modules)

        # send email notification
        if(delta_text != ""): # and prev_data != {}
            send_mail(f"Studenthub credits changed! {prev_credits} -> {curr_credits}", delta_text)
        else:
            print(f"No credits change <{credits_text}>")

        # rewrite new data to dump
        open("studenthub.json", "w").write(json.dumps(data))

        # close browser session
        driver.quit()
        if(repeat != 1):
            break

        # configure intervall, between 1h and 2h
        delay_h = 1 # 60*60
        delay_h += random.randint(1800, 3600)
        print(f"waiting {round(delay_h / 60)} minutes")
        time.sleep(delay_h)
        retry = 0
    except Exception as e:
        print(e)
        last_error = str(e)
        if(retry >= max_retry - 1):
            print("max retry reached, aborting")
            break

        print("retrying")
        retry += 1

No credits change <84/180 Credits>
waiting 58.68333333333333 minutes


In [ ]:
# send abort mail
send_mail(f"Studenthub notifier aborted!", f"Error occured:\r\n{last_error}")


---
Spielwiese

In [ ]:
# send slack notification (not working right now)
# if(delta_text != ""):
#     slack.send_notify(slack_channel, username=slack_username, text=f'@studenthubnotificatio {delta_text}')